In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [44]:
# Read Data
train = pd.read_excel(r"/content/drive/MyDrive/深度學習於醫學影像/train data/data.xlsx")
test = pd.read_excel(r"/content/drive/MyDrive/深度學習於醫學影像/test data/test.xlsx")

In [13]:
test

,ID,Filename,主動脈硬化(鈣化),動脈彎曲,肺野異常,肺紋增加,脊椎病變,肺間肋膜增厚,心臟肥大
0,TDR0120171108100516,A0_28.dcm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TDR0120171130152723,A0_9.dcm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TDR0120171202104940,A0_8.dcm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TDR0120171202111823,A0_7.dcm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TDR0120171207092217,A0_6.dcm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
95,TDR0520180226145555,220_173.dcm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,TDR0520180226145713,220_169.dcm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,TDR0520180226154656,220_168.dcm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,TDR0520180227153750,220_163.dcm,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Dummy Lable Dataframe
count = list(range(0,800))
df_label = pd.DataFrame(columns=['主動脈硬化(鈣化)','動脈彎曲','肺野異常','肺紋增加','脊椎病變','肺尖肋膜增厚','心臟肥大'],index=count)
df_label = df_label.fillna(0)
# Dummy Lable
# '主動脈硬化(鈣化)','動脈彎曲','肺野異常','肺紋增加','脊椎病變','肺間肋膜增厚','心臟肥大'
for i in range(800):
    #print(i,train['label'][i])
    row_label = train['label'][i]
    if row_label == 'normal':
        continue
    if '主動脈硬化(鈣化)' in row_label:
        df_label['主動脈硬化(鈣化)'][i] = 1
    if '肺尖肋膜增厚' in row_label:
        df_label['肺尖肋膜增厚'][i] = 1
    if '動脈彎曲' in row_label:
        df_label['動脈彎曲'][i] = 1
    if '肺野異常' in row_label:
        df_label['肺野異常'][i] = 1
    if '肺紋增加' in row_label:
        df_label['肺紋增加'][i] = 1
    if '脊椎病變' in row_label:
        df_label['脊椎病變'][i] = 1
    if '心臟肥大' in row_label:
        df_label['心臟肥大'][i] = 1

In [15]:
df_label

,主動脈硬化(鈣化),動脈彎曲,肺野異常,肺紋增加,脊椎病變,肺尖肋膜增厚,心臟肥大
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
795,0,1,0,0,1,0,0
796,0,1,0,0,1,0,0
797,0,1,0,0,1,0,0
798,0,1,0,0,1,0,0


In [ ]:
# dicom to jpg
import cv2 as cv
import numpy as np
import pydicom as PDCM

def Dicom_to_Image(Path):
    DCM_Img = PDCM.read_file(Path)

    rows = DCM_Img.get(0x00280010).value #Get number of rows from tag (0028, 0010)
    cols = DCM_Img.get(0x00280011).value #Get number of cols from tag (0028, 0011)

    Instance_Number = int(DCM_Img.get(0x00200013).value) #Get actual slice instance number from tag (0020, 0013)

    Window_Center = int(DCM_Img.get(0x00281050).value) #Get window center from tag (0028, 1050)
    Window_Width = int(DCM_Img.get(0x00281051).value) #Get window width from tag (0028, 1051)

    Window_Max = int(Window_Center + Window_Width / 2)
    Window_Min = int(Window_Center - Window_Width / 2)

    if (DCM_Img.get(0x00281052) is None):
        Rescale_Intercept = 0
    else:
        Rescale_Intercept = int(DCM_Img.get(0x00281052).value)

    if (DCM_Img.get(0x00281053) is None):
        Rescale_Slope = 1
    else:
        Rescale_Slope = int(DCM_Img.get(0x00281053).value)

    New_Img = np.zeros((rows, cols), np.uint8)
    Pixels = DCM_Img.pixel_array

    for i in range(0, rows):
        for j in range(0, cols):
            Pix_Val = Pixels[i][j]
            Rescale_Pix_Val = Pix_Val * Rescale_Slope + Rescale_Intercept

            if (Rescale_Pix_Val > Window_Max): #if intensity is greater than max window
                New_Img[i][j] = 255
            elif (Rescale_Pix_Val < Window_Min): #if intensity is less than min window
                New_Img[i][j] = 0
            else:
                New_Img[i][j] = int(((Rescale_Pix_Val - Window_Min) / (Window_Max - Window_Min)) * 255) #Normalize the intensities

    return New_Img, Instance_Number

def main():
    # Input Path
    input_path = '/content/drive/MyDrive/深度學習於醫學影像/train data/'
    for i in range(800):
      print(i,"start")
      Input_Image = input_path + train['category'][i] + '/' + train['Filename'][i]
      #Input_Image = '/content/drive/MyDrive/深度學習於醫學影像/train data/normal/220.dcm'

      Output_Image, Instance_Number = Dicom_to_Image(Input_Image)

      cv.imwrite('/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/'+train['Filename'][i]+'.jpg', Output_Image)
      print(i,"finished")

if __name__ == "__main__":
    main()

In [18]:
# Read images and merge them into the training data X.
X = np.array([])
path = '/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/'
for i in range(800):
  print(i)
  input_path = path + train['Filename'][i] + '.jpg'
  print(input_path)
  # Load image and Resize into (224, 224) 
  img = image.load_img(input_path, target_size=(224, 224))
  # Show image
  #imgplot = plt.imshow(img)
  #plt.show()
  # Add one dimension and Become to (1, 224, 224, 3), which the last dimension is color.
  img2 = image.img_to_array(img)
  img2 = np.expand_dims(img2, axis=0)
  #print(img2.shape)
  if len(X.shape) == 1:
      X = img2
  else:
      # Combine the pixels of all images
      X = np.concatenate((X, img2), axis=0)

X = preprocess_input(X)
    
np.save('x_ray', X)

0
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_243 (2).dcm.jpg
1
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_97.dcm.jpg
2
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/4440_39.dcm.jpg
3
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_273 (2).dcm.jpg
4
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_96.dcm.jpg
5
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_95.dcm.jpg
6
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/4440_38.dcm.jpg
7
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_272 (2).dcm.jpg
8
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_363.dcm.jpg
9
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_271 (2).dcm.jpg
10
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_76.dcm.jpg
11
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/220_94.dcm.jpg
12
/content/drive/MyDrive/深度學習於醫學影像/train data/train_jpg/4440_37.dcm.jpg
13
/content/drive/MyDrive/深度學習於醫學影像/train data/tra

NameError: ignored

In [19]:
from tensorflow.keras.applications.vgg16 import preprocess_input
X = preprocess_input(X)

In [20]:
X.shape

(800, 224, 224, 3)

In [21]:
y = np.array(df_label)

In [29]:
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, outputs=output)
for layer in restnet.layers:
    layer.trainable = False
restnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [35]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model = Sequential()
model.add(restnet)
model.add(Dense(512, activation='relu', input_dim=3))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(7, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Functional)         (None, 100352)            23587712  
_________________________________________________________________
dense_6 (Dense)              (None, 512)               51380736  
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 7)                 3591      
Total params: 75,234,695
Trainable params: 51,646,983
Non-trainable params: 23,587,712
_________________________________

In [60]:
model.fit(X, y, epochs=10, validation_split=0.2, verbose=2)

Epoch 1/10
20/20 - 126s - loss: 0.1084 - accuracy: 0.2547 - val_loss: 1.0438 - val_accuracy: 0.0688
Epoch 2/10
20/20 - 126s - loss: 0.1130 - accuracy: 0.2250 - val_loss: 1.0074 - val_accuracy: 0.1000
Epoch 3/10
20/20 - 127s - loss: 0.1009 - accuracy: 0.2656 - val_loss: 0.9241 - val_accuracy: 0.0250
Epoch 4/10
20/20 - 127s - loss: 0.1000 - accuracy: 0.2469 - val_loss: 1.0456 - val_accuracy: 0.0188
Epoch 5/10
20/20 - 127s - loss: 0.0950 - accuracy: 0.2469 - val_loss: 1.0110 - val_accuracy: 0.0375
Epoch 6/10
20/20 - 127s - loss: 0.0850 - accuracy: 0.3266 - val_loss: 1.0478 - val_accuracy: 0.0875
Epoch 7/10
20/20 - 128s - loss: 0.0939 - accuracy: 0.2406 - val_loss: 0.9828 - val_accuracy: 0.0437
Epoch 8/10
20/20 - 127s - loss: 0.0837 - accuracy: 0.2734 - val_loss: 1.0312 - val_accuracy: 0.0500
Epoch 9/10
20/20 - 127s - loss: 0.0924 - accuracy: 0.2313 - val_loss: 0.9853 - val_accuracy: 0.0625
Epoch 10/10
20/20 - 127s - loss: 0.0914 - accuracy: 0.2875 - val_loss: 1.0685 - val_accuracy: 0.1000

In [69]:
#/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpg10_0 (2).dcm.jpg
test_path = '/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpg'    
for n in range(100):
  output_path = test_path + test['Filename'][n]+ '.jpg'
  print(output_path)
  img = image.load_img(output_path, target_size=(224, 224))
  #imgplot = plt.imshow(img)
  #plt.show()
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  model_matrix = model.predict(x)
  print("n = ",n)
  print(model_matrix)
  for i in range(7):
    if model_matrix[0][i] > 0.1:
      if i == 0:
        test['主動脈硬化(鈣化)'][n] = '✔'
      elif i == 1:
        test['動脈彎曲'][n] = '✔'
      elif i == 2:
        test['肺野異常'][n] = '✔'
      elif i == 3:
        test['肺紋增加'][n] = '✔'
      elif i == 4:
        test['脊椎病變'][n] = '✔'
      elif i == 5:
        test['肺間肋膜增厚'][n] = '✔'
      elif i == 6:
        test['心臟肥大'][n] = '✔'

/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpgA0_28.dcm.jpg
n =  0
[[0.00107834 0.00068682 0.00051764 0.00195879 0.00206357 0.00206783
  0.00044882]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpgA0_9.dcm.jpg
n =  1
[[0.01475474 0.0007346  0.00102565 0.00256222 0.00119242 0.00344014
  0.00122032]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpgA0_8.dcm.jpg
n =  2
[[0.00208956 0.00092304 0.00072658 0.00010353 0.00090709 0.00072649
  0.00085533]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpgA0_7.dcm.jpg
n =  3
[[0.00069192 0.00036252 0.00040421 0.00060636 0.00028712 0.00034735
  0.00023749]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpgA0_6.dcm.jpg
n =  4
[[0.02919233 0.00067157 0.00028548 0.02334183 0.0038496  0.0013192
  0.00354245]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpgA0_1.dcm.jpg
n =  5
[[0.14272323 0.01091206 0.0112918  0.04209703 0.01555791 0.02177659
  0.01570839]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpgA0_2.dcm.jpg


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


n =  6
[[0.02225611 0.00989023 0.00444525 0.00696561 0.01024848 0.00836796
  0.0043447 ]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpgA0_0.dcm.jpg
n =  7
[[0.23609257 0.02428788 0.01007837 0.12090883 0.05216858 0.07009608
  0.0155209 ]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpg220_207 (2).dcm.jpg


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


n =  8
[[3.4096837e-04 4.5294928e-05 8.9708892e-06 1.8742681e-04 1.3056397e-04
  1.9419911e-05 2.4482528e-05]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpg220_205 (2).dcm.jpg
n =  9
[[1.4612079e-04 1.8635392e-04 4.0296785e-05 1.9490719e-04 6.7749619e-04
  1.0654628e-03 3.5406600e-05]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpg220_204 (2).dcm.jpg
n =  10
[[1.69545412e-04 6.33298187e-05 3.00794840e-04 1.06303756e-04
  6.92457761e-05 3.61859798e-04 4.93713378e-05]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpg220_202 (2).dcm.jpg
n =  11
[[0.0037016  0.00207892 0.00174075 0.00402513 0.00426584 0.0026195
  0.0003019 ]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpg220_200 (2).dcm.jpg
n =  12
[[0.02882817 0.0026381  0.00065875 0.00945368 0.00204349 0.00282297
  0.00884315]]
/content/drive/MyDrive/深度學習於醫學影像/test data/test_jpg220_196 (2).dcm.jpg
n =  13
[[0.00124723 0.00049198 0.00011311 0.00021303 0.00097945 0.00051609
  0.00017387]]
/content/drive/MyDrive/深度學習於醫學影像

In [70]:
# Save VGG16 result
test.to_csv("/content/drive/MyDrive/深度學習於醫學影像/ResNet50.csv")  